# Merge the data scraped from Zoocasa and Map
The data were collected separtely:
1. Scrap through all the Zoocasa listings
2. Scrap through individual listing to get the parking, type details 
3. Scrap Map to find the longitude and latitude

In [1]:
import pandas as pd
import numpy as np
import json

In [199]:
file1 = r'C:\Users\Penelope\Desktop\web-scraper\SP-500.json'
with open(file1) as train_file1:
    dict_train1 = json.load(train_file1)

df1 = pd.DataFrame.from_dict(dict_train1, orient='index')
df1 = df1.T
df1.head(3)
print(len(df1))

file2 = r'C:\Users\Penelope\Desktop\web-scraper\SP-1500.json'
with open(file2) as train_file2:
    dict_train2 = json.load(train_file2)

df2 = pd.DataFrame.from_dict(dict_train2, orient='index')
df2 = df2.T
print(len(df2))

14000
27959


In [225]:
df = pd.concat([df1, df2], axis = 0).reset_index(drop=True)
print(len(df))
df.drop_duplicates(subset='web_link', keep='first', inplace=True)
df = df.reset_index()
print(len(df))

41959
41010


In [226]:
NOTO = [] # notice there are non-Toronto units 
for i in range(len(df)):
    if 'toronto-on-real-estate' not in df.loc[i, 'web_link'].split('/')[1]: 
        NOTO.append(i)
df.drop(labels=NOTO, axis=0, inplace=True)
print(len(df))

39390


In [227]:
detail_file1 = r'C:\Users\Penelope\Desktop\web-scraper\type-t-500.json'
with open(detail_file1) as train_file1:
    dict_detail1 = json.load(train_file1)
detaildf1 = pd.DataFrame.from_dict(dict_detail1, orient='index')
detaildf1 = detaildf1.T
print(len(detaildf1))

detail_file2 = r'C:\Users\Penelope\Desktop\Scrapper\type-t-1500.json'
with open(detail_file2) as train_file2:
    dict_detail2 = json.load(train_file2)
detaildf2 = pd.DataFrame.from_dict(dict_detail2, orient='index')
detaildf2 = detaildf2.T
print(len(detaildf2))

14000
27959


In [228]:
df_detail = pd.concat([detaildf1, detaildf2], axis = 0).reset_index(drop=True)
print(len(df_detail))
df_detail.drop_duplicates(subset='link', keep='first', inplace=True)
print(len(df_detail))
df_detail = df_detail.dropna(how='any',axis=0)
print(len(df_detail))

41959
41010
40993


In [229]:
gmap_file1 = r'C:\Users\Penelope\Desktop\web-scraper\gmap-intermediate-firstpart0-13500.json'
with open(gmap_file1) as gmap_file1:
    dict_gmap1 = json.load(gmap_file1)
gmapdf1 = pd.DataFrame.from_dict(dict_gmap1, orient='index')
gmapdf1 = gmapdf1.T
print(len(gmapdf1))


gmap_file2 = r'C:\Users\Penelope\Desktop\Scrapper\gmap-intermediate-secondpart0-8500.json'
with open(gmap_file2) as gmap_file2:
    dict_gmap2 = json.load(gmap_file2)
gmapdf2 = pd.DataFrame.from_dict(dict_gmap2, orient='index')
gmapdf2 = gmapdf2.T
print(len(gmapdf2))

gmap_file3 = r'C:\Users\Penelope\Desktop\Scrapper\gmap-intermediate-secondpart8501-27500.json'
with open(gmap_file3) as gmap_file3:
    dict_gmap3 = json.load(gmap_file3)
gmapdf3 = pd.DataFrame.from_dict(dict_gmap3, orient='index')
gmapdf3 = gmapdf3.T
print(len(gmapdf3))

13501
8501
19000


In [236]:
gmapdf = pd.concat([gmapdf1, gmapdf2, gmapdf3], axis=0).reset_index()
print(len(gmapdf))
gmapdf.drop_duplicates(subset='address',keep='first', inplace=True)
print(len(gmapdf))
gmapdf = gmapdf.dropna(how='any',axis=0)
print(len(gmapdf))
gmapdf=gmapdf.reset_index()

41002
40200
40200


In [237]:
LAT, LONG = [[] for i in range(2)]
for i in range(len(gmapdf)):
    la, lo = gmapdf.loc[i, 'listing_gmap_urls'].split('@')[1].split(',')[0:2]
    LAT.append(la)
    LONG.append(lo)
    
gmapdf['lat'] = LAT
gmapdf['long'] = LONG

In [247]:
print(len(df))
print(len(df_detail))
data = pd.merge(df, df_detail, how='inner', left_on='web_link', right_on='link')
data = pd.merge(data, gmapdf, how='inner', left_on='address', right_on='address')
print(len(data))

39390
40993
38891


In [248]:
data.head(3)

,index_x,sold_price,list_price,address,web_link,bedroom,bathroom,sqft,link,type,level,parking,level_0,index_y,listing_gmap_urls,lat,long
0,0,"$1,680,000","List Price$1,499,000",30 Norway Ave,/toronto-on-real-estate/30-norway-ave,3 Bed,3 Bath,N/A sqft,/toronto-on-real-estate/30-norway-ave,TypeSemi-Detached,Levels2-Storey,1,0,0,https://www.google.com/maps/place/30+Norway+Av...,43.6726534,-79.3146166
1,1,"$600,000","List Price$550,000",1403 - 76 Shuter St,/toronto-on-real-estate/1403-76-shuter-st,1 Bed,1 Bath,0-499 sqft,/toronto-on-real-estate/1403-76-shuter-st,TypeCondo Apt,LevelsApartment,1,1,1,https://www.google.com/maps/place/76+Shuter+St...,43.6552933,-79.3841171
2,2,"$660,000","List Price$599,000",103 - 80 Marine Parade Dr,/toronto-on-real-estate/103-80-marine-parade-dr,1 Bed,1 Bath,500â€“599 sqft,/toronto-on-real-estate/103-80-marine-parade-dr,TypeCondo Apt,LevelsApartment,1,2,2,https://www.google.com/maps/place/80+Marine+Pa...,43.6261877,-79.4860442


In [249]:
data = data.drop(['index_x', 'index_y'], axis=1)
data.head(3)

,sold_price,list_price,address,web_link,bedroom,bathroom,sqft,link,type,level,parking,level_0,listing_gmap_urls,lat,long
0,"$1,680,000","List Price$1,499,000",30 Norway Ave,/toronto-on-real-estate/30-norway-ave,3 Bed,3 Bath,N/A sqft,/toronto-on-real-estate/30-norway-ave,TypeSemi-Detached,Levels2-Storey,1,0,https://www.google.com/maps/place/30+Norway+Av...,43.6726534,-79.3146166
1,"$600,000","List Price$550,000",1403 - 76 Shuter St,/toronto-on-real-estate/1403-76-shuter-st,1 Bed,1 Bath,0-499 sqft,/toronto-on-real-estate/1403-76-shuter-st,TypeCondo Apt,LevelsApartment,1,1,https://www.google.com/maps/place/76+Shuter+St...,43.6552933,-79.3841171
2,"$660,000","List Price$599,000",103 - 80 Marine Parade Dr,/toronto-on-real-estate/103-80-marine-parade-dr,1 Bed,1 Bath,500â€“599 sqft,/toronto-on-real-estate/103-80-marine-parade-dr,TypeCondo Apt,LevelsApartment,1,2,https://www.google.com/maps/place/80+Marine+Pa...,43.6261877,-79.4860442


In [250]:
data.tail(4)

,sold_price,list_price,address,web_link,bedroom,bathroom,sqft,link,type,level,parking,level_0,listing_gmap_urls,lat,long
38887,"$712,500","List Price$499,000",61 - 28 Livingston Rd,/toronto-on-real-estate/61-28-livingston-rd,3 Bed,3 Bath,1400â€“1599 sqft,/toronto-on-real-estate/61-28-livingston-rd,TypeAtt/Row/Twnhouse,Levels3-Storey,2,40969,https://www.google.com/maps/place/28+Livingsto...,43.7414817,-79.2073021
38888,"$1,635,000","List Price$1,398,000",33 Unicorn Ave,/toronto-on-real-estate/33-unicorn-ave,4 Bed,4 Bath,N/A sqft,/toronto-on-real-estate/33-unicorn-ave,TypeDetached,Levels1 1/2 Storey,3,40983,https://www.google.com/maps/place/33+Unicorn+A...,43.7927947,-79.3920482
38889,"$760,000","List Price$699,000",27 - 749 Steeles Ave W,/toronto-on-real-estate/27-749-steeles-ave-w,3 Bed,3 Bath,1200â€“1399 sqft,/toronto-on-real-estate/27-749-steeles-ave-w,TypeCondo Apt,LevelsApartment,0,40985,https://www.google.com/maps/place/749+Steeles+...,43.7930226,-79.4492351
38890,"$551,000","List Price$525,000",1702 - 350 Seneca Hill Dr,/toronto-on-real-estate/1702-350-seneca-hill-dr,2 Bed,1 Bath,800â€“899 sqft,/toronto-on-real-estate/1702-350-seneca-hill-dr,TypeSemi-Detached,LevelsBacksplit 5,3,40988,https://www.google.com/maps/place/350+Seneca+H...,43.7907653,-79.3648767


In [253]:
data.to_csv('./dataset/house_listing_complete_2022.csv')